In [ ]:
%run
./01_data_generation

In [ ]:
from src.data_cleaning import clean_retail_data
from src.feature_engineering import add_total_sales

df = clean_retail_data(df)
df = add_total_sales(df)

df.info()


In [ ]:
spark_df = spark.createDataFrame(df)
spark_df.createOrReplaceTempView("retail_sales")

In [ ]:
%sql
SELECT Product,
       SUM(TotalSales) AS TotalRevenue
FROM retail_sales
GROUP BY Product
ORDER BY TotalRevenue DESC
LIMIT 5

In [ ]:
%sql
SELECT Region,
       SUM(TotalSales) AS RegionalSales
FROM retail_sales
GROUP BY Region
ORDER BY RegionalSales DESC

In [ ]:
%sql
SELECT DATE_FORMAT(Date, 'yyyy-MM') AS Month,
       SUM(TotalSales) AS MonthlyRevenue
FROM retail_sales
GROUP BY DATE_FORMAT(Date, 'yyyy-MM')
ORDER BY Month

In [ ]:
%sql
SELECT Product,
       Region,
       SUM(TotalSales) AS ProductRegionSales,
       RANK() OVER (PARTITION BY Region ORDER BY SUM(TotalSales) DESC) AS RankInRegion
FROM retail_sales
GROUP BY Product, Region